In [1]:
'''
GLOBAL ALIGNMENT WITH AFFINE GAPS
'''

def read_single_fasta(filename):
    """
    Reads in a single fasta file (only contains one sequence) and returns
    the sequence from the file.
    Input: filename (String) - the name of the fasta file to read. The
    sequence may be on multiple lines in the fasta file.
    Output: sequence (String) - a single string representing the sequence in
    the fasta file.
    Code from HW1 solutions
    """
    f = open(filename, 'r')
    sequence = ""
    for line in f:
        if not line.startswith(">"):
            #only include non headers
            sequence += line.rstrip()
    f.close()
    sequence = sequence.upper()
    return sequence
'''
READING PENALTY FILE
'''

def print_file(filename):
    '''
    This function just prints to the screen all contents within a file.
    Input: filename (String) - the name of the file to print.
    '''
    file = open(filename, 'r') # 'r' says we are reading the file

    print("Printing contents of following file:",filename)
    for line in file: # this is an easy way to iterate through a file
        line = line.rstrip() # all lines have newline character at the end.
        print(line)
    file.close() #always close the file after opening it

    print("\n\n\n") #print a few lines of white space


def read_file(filename):
    '''
    This function takes in a filename and returns a tuple
    containing the four different scoring parameters.
    Input: filename (string) - the name of the file to open.
    Return: scores (tuple of ints) - this tuple of length 4
            contains the following values in the following order:
            match score, mismatch score, gap-extend, gap-open
    '''
    file = open(filename, 'r') # 'r' says we are reading the file

    file.readline() #skip past header line

    line = file.readline() #reads next line, returns a string.
    line = line.rstrip() #remove all white space, return characters, etc
                         #on right hand side of string.

    file.close() #done with file, so close it

    vals = line.split("\t") #turn string into list of items originally
                            #separated by tabs

    scores = tuple(vals) #convert from list (mutable) to tuple (immutable)

    return scores

def write_file(scores, filename):
    '''
    This function takes in a tuple of scores and a filename
    and saves those scores to file as a new scoring file.
    Input: scores (tuple of ints) - a tuple of length 4 containing match,
            mismatch, gap-open and gap extend values.
           filename (String) - the name of the file to create.
    Return: None
    '''
    file = open (filename, 'w') #w indicates we are writing to a file
    header = "#match\tmismatch\tgap-extend\tgap-open\n" #need to add newline at end
    file.write(header) #This adds header to the file

    #Build string of tab separated scores.
    vals = "\t".join(scores) #creates a tab sep string of everything in scores.
    vals = vals + "\n" #add newline at end
    file.write(vals) #This adds vals to the file
    file.close() #You have to close a file in order for everything you wrote
                 #into it to show up
        
#READS FILE TO GET PENALTIES
def scoringfile(scfilename):
    '''
    Read in a scoring file provided as first command line parameter.
    Print scores to the screen.  Then return the final values as 1 string seperates by semicolons.
    '''
    arg = scfilename
    
    num_args = len(arg) - 1 #retrieve number of command line arguments
    if (num_args < 1):
        print("Program requires a file name.")
        

    filename = arg #first argument should be name of scoring file.
                           #this assumes file is in same directory as this code.

    #print contents of file to screen
    print_file(filename)

    #print all scores to the screen
    scores = read_file(filename)
    print("Scores contained in:",filename)
    count = 0
    for v in scores:
        #print("Score:", v)
        if count ==0:
            match = int(v)
        elif count == 1:
            mismatch = int(v)
        elif count == 2:
            gapex = int(v)
        elif count == 3:
            gapop = int(v)
        count += 1
        
    print("MATCH: ", match, " MISMATCH: ", mismatch, " GAPEX: ", gapex, " GAPOP: ", gapop)

    fin = str(match)+ ";"+str(mismatch)+";"+ str(gapex)+";"+ str(gapop)
    print(fin)
    print()
    return fin
'''
#GLOBAL ALIGNMENT WITH AFFINE GAPS
#This function uses a dynamic programming algorithm to calculate the optimal
#alignment score between two strings using gap, mismatch, and match scores
#Parameters: s1 and s2 are two strings (do not have to be the same length)
#            gap, mismatch, and match are all integers, gap and mismatch are
#            typically negative values, match is typically a positive value
#Returns: the optimal global alignment score, as we as a 2-D list of hints
#         to help determine the actual alignment
'''
def globalAF(s1, s2, gapex, mismatch, match, gapop):
    m = len(s1)
    n = len(s2)

    #Initialize 2 2-d lists of size (m+1)rows * (n+1)columns
    gapy = [[0 for i in range(n+1)] for j in range(m+1)]  #gapy
    matches = [[0 for i in range(n+1)] for j in range(m+1)]   #matches
    gapx = [[0 for i in range(n+1)] for j in range(m+1)]   #gapx
    
    matrix = [[0 for i in range(n+1)] for j in range(m+1)]   #final/optimal matrix
    
    hints = [[0 for i in range(n+1)] for j in range(m+1)]   #backtracking hints
    
    gapex = int(gapex)
    gapop = int(gapop)
    
    #Below is the pseudocode for computing the alignment score table (matrix)
    #Once you get this working, add in code to keep track of the hints
    #You'll see in get_alignment, that it is expecting you to present hints as:
    # \\ (diagonal), | (from above), - (from left)
    
    #initalization of matrices        
    for i in range(0, m+1): #first row
        matrix[i][0] = gapop + (gapex * i)
        gapy[i][0] = gapop + (gapex * i)
        
        #invalid
        gapx[i][0] = -999999999999
        matches[i][0] = -9999999999
        
        hints[i][0] = "|"

    for j in range(0, n+1):
        matrix[0][j] = gapop + (gapex * j)
        gapx[0][j] = gapop + (gapex * j)
        
        #invalid
        gapy[0][j] = -999999999999
        matches[0][j] = -9999999999
        
        hints[0][j] = "-"

    #initalize the [0][0] corner
    matrix[0][0] = 0  
    matches[0][0] = 0
    gapy[0][0] = gapop 
    gapx[0][0] = gapop
    
    #new gap (gapop + gapex) if coming from match/mismatch
    #other wise extend (gapex)
    for i in range(1, m+1):
        for j in range(1, n+1):
            #x and y match/mismatch
            if s1[i-1] == s2[j-1]:
                score1_1 = matches[i-1][j-1] #prev match
                score1_2 = gapy[i-1][j-1] #prev gap in y direction
                score1_3 = gapx[i-1][j-1] #prev gap in x direction
                score1 = max(matches[i-1][j-1], gapy[i-1][j-1], gapx[i-1][j-1]) + match
                matches[i][j] = score1
                #print("MAX: ", max(matches[i-1][j-1], gapy[i-1][j-1], gapx[i-1][j-1]))
            else:
                score1 = max(matches[i-1][j-1], gapy[i-1][j-1], gapx[i-1][j-1]) + mismatch
                matches[i][j] = score1
            
            #gap in the x direction
            score2_1 = gapex + gapop + matches[i][j-1] 
            score2_2 = gapex + gapx[i][j-1]
            score2_3 = gapex + gapop + gapy[i][j-1]
            
            score2 = max(score2_1, score2_2, score2_3)
            gapx[i][j] = score2
            
            #gap in the y direction
            score3_1 = gapex + gapop + matches[i-1][j]
            score3_2 = gapex + gapop + gapx[i-1][j]
            score3_3 = gapex + gapy[i-1][j]
            
            score3 = max(score3_1, score3_2, score3_3)
            gapy[i][j] = score3
            
            
            #max score overall
            maxScore = max(score1, score2, score3)
            
            #add to optimal matrix and add direction to hints
            matrix[i][j] = maxScore
            if maxScore == score1:
                hints[i][j] = "\\"
            elif maxScore == score2:
                hints[i][j] = "-"
            else:
                hints[i][j] = "|"
            
#     print("matrix /S")
#     for row in matrix:
#         print(row)
#     print("gap y /F")
#     for row in gapy:
#         print(row)
#     print("gap x /E")
#     for row in gapx:
#         print(row)
#     print("matches / G")
#     for row in matches:
#         print(row)
#     print("hints")
#     for row in hints:
#         print(row)
    return matrix[m][n], hints

#This function takes in 2 strings and a 2-D list of backtracking hints
#and returns the global alignment of the strings
#Parameters: A and B are two strings (do not have to be the same length)
#            hints is a 2-D lists containing \\, -, or | symbols
#Returns: two strings describing the global alignment between A and B

#requires 2 sequences, prints alignment and scores
def globalalignAF(seq1, seq2):
    s1 = seq1
    s2 = seq2
    option = 0
    opvalid = False
    while not opvalid:
        option = input("For penalties, Filename (1) or by hand (2)? ")
        if option == "1":
            filename = input("Filename: ")
            scores = str(scoringfile(filename))
            match, mismatch, gapex, gapop = scores.split(";")
            gap_score = int(gapex)
            mismatch_score = int(mismatch)
            match_score = int(match)
            gapop = int(gapop)
            opvalid = True
        elif option == "2":
            mismatch_score = int(input("Mismatch: "))
            match_score = int(input("Match: "))
            gapop = int(input("Gap open: "))
            gap_score = int(input("Gap extend: "))
            opvalid = True
        else:
            option = input("Try again. Filename (1) or by hand (2)? ")
            opvalid = False
    
    score, hints = globalAF(s1, s2, gap_score, mismatch_score, match_score, gapop)
    print("The global alignment with afine gaps score is", score)
    
    align1, align2 = get_alignment(s1, s2, hints)
    print(align1)
    print(align2)
   

In [6]:
'''LOCAL ALIGNMENT'''

'''
#This function uses a dynamic programming algorithm to calculate the optimal
#alignment score between two strings using gap, mismatch, and match scores
#Parameters: s1 and s2 are two strings (do not have to be the same length)
#            gap, mismatch, and match are all integers, gap and mismatch are
#            typically negative values, match is typically a positive value
#Returns: the optimal local alignment score, as we as a 2-D list of hints
#         to help determine the actual alignment
'''
def SmithWaterman(s1, s2, gap, mismatch, match, gapop):
    m = len(s1)
    n = len(s2)

    # Initialize 2 2-d lists of size (m+1)rows * (n+1)columns
    matrix = [[0 for i in range(n + 1)] for j in range(m + 1)]  # scores
    hints = [[0 for i in range(n + 1)] for j in range(m + 1)]  # backtracking hints

    # \\ (diagonal), | (from above), - (from left)

    matrix[0][0] = 0
    for i in range(1, m+1):
        matrix[i][0] = 0
        hints[i][0] = "|"

    for j in range(1, n+1):
        matrix[0][j] = 0
        hints[0][j] = "-"

    # checks the recurrences
    for i in range(1, m+1):
        for j in range(1, n+1):
            if s1[i-1] == s2[j-1]:
                score1 = matrix[i-1][j-1] + match
            else:
                score1 = matrix[i-1][j-1] + mismatch

            # all the negative values become 0
            if score1 < 0:
                score1 = 0

            score2 = matrix[i][j-1] + gap
            if score2 < 0:
                score2 = 0

            score3 = matrix[i-1][j] + gap
            if score3 < 0:
                score3 = 0

            maxScore = max(score1, score2, score3)
            matrix[i][j] = maxScore

            if maxScore == score1:
                hints[i][j] = "\\"
            elif maxScore == score2:
                hints[i][j] = "-"
            elif maxScore == score3:
                hints[i][j] = "|"
            else: # meaning that the score is 0
                hints[i][j] = "."
                
    return matrix[m][n], hints


def get_alignment(A, B, hints):
    i = len(A)
    j = len(B)
    align1 = ''
    align2 = ''

    while i > 0 or j > 0:
        if hints[i][j] == '\\':
            align1 = A[i - 1] + align1
            align2 = B[j - 1] + align2
            i -= 1
            j -= 1
        elif hints[i][j] == '|':
            align1 = A[i - 1] + align1
            align2 = "-" + align2
            i -= 1
        elif hints[i][j] == '.':
            break
        else:
            align1 = "-" + align1
            align2 = B[j - 1] + align2
            j -= 1

    print(i, j)
    return align1, align2

def localalign(seq1, seq2):
    s1 = seq1
    s2 = seq2
    option = 0
    while option != "1" and option != "2":
        option = input("For penalties, Filename (1) or by hand (2)? ")
        if option == "1":
            filename = input("Filename: ")
            scores = str(scoringfile(filename))
            match, mismatch, gapex, gapop = scores.split(";")
            gap_score = int(gapex)
            mismatch_score = int(mismatch)
            match_score = int(match)
            gapop = int(gapop)
        elif option == "2":
            mismatch_score = int(input("Mismatch: "))
            match_score = int(input("Match: "))
            gapop = int(input("Gap open: "))
            gap_score = int(input("Gap extend: "))
        else:
            option = input("Try again. Filename (1) or by hand (2)? ")
    
    score, hints = SmithWaterman(s1, s2, gap_score, mismatch_score, match_score, gapop)
    print("The local alignment is", score)
    
    align1, align2 = get_alignment(s1, s2, hints)
    print(align1)
    print(align2)

In [8]:
'''
HUMAN/CAT/RAT genome comparisons
'''

cat = read_single_fasta("TP53-Cat.fasta")
human = read_single_fasta("TP53-Human.fasta")
rat = read_single_fasta("TP53-Rat.fasta")

print("CAT/HUMAN")
globalalignAF(cat, human)
localalign(cat, human)
print("CAT/RAT")
globalalignAF(cat, rat)
localalign(cat, rat)
print("RAT/HUMAN")
globalalignAF(rat, human)
localalign(rat, human)

CAT/HUMAN


For penalties, Filename (1) or by hand (2)?  1
Filename:  BRCAscoreA.txt


Printing contents of following file: BRCAscoreA.txt
#match	mismatch	gap-extend	gap-open
1	-1	-1	-1




Scores contained in: BRCAscoreA.txt
MATCH:  1  MISMATCH:  -1  GAPEX:  -1  GAPOP:  -1
1;-1;-1;-1

The global alignment with afine gaps score is -26
0 0
TTAATGAAATGATACACGTCTTGCCCCCAAACAGTTGTTTTATTACACGATGTCTGAGACTTGGTGCCTTCACGGGT-GCTGATTCCCTAACGTC--CCTGTTG-GTGGATGGGATGATCAAGTTCCTACGACC-GGGACAGCTGGTCAGGGGTCCCCTCCTTGCTGGCCCGGCCGAATC-CTGTCTTCCCTCTCGCCGAACCTC-ACTTCCTAAAAGGAAGTCTGACTCTGTCCTACGCCCGCGAGGGATCCGATCCCTG-GTGTAGGATCTGTATCTATCAAGGTCTGTTCC-CTGCCCCCGACTCCTCAGGATCAATTTCTTTTGCACTCTTGGAGGCACATCTGCGTATTTCCCCACACCCTT--CC-CC--CT--C--------TCCCCTTTTTATATC---C-CCT-TTTT---AT---A------T-----C-G-GTTTC------GT-
--CA-GG-CTG-GAGTGGAG-TGGCGTG-ATC--TTG---GCTT--AC--TG-C--AGCCTT--TGCCTCCCCGGCTCGAGCAGT-CCT-GCCTCAGCCTCCGGAGTAGCTGGGACCA-CAGGTTCATGCCACCATGGCCAGC----CAACTTTTGCATGTTTTGTAGAGATGGGGTCTCACAGTGTTGCC-CAGGCTG-GTCTCAAACTCCT----GG--G-CTCAGGCGATCC-A--CCTGTCTCAG--CC--TCCCAGAGTGCTGG----G-AT-TA-CAA-TTGT

For penalties, Filename (1) or by hand (2)?  1
Filename:  BRCAscoreA.txt


Printing contents of following file: BRCAscoreA.txt
#match	mismatch	gap-extend	gap-open
1	-1	-1	-1




Scores contained in: BRCAscoreA.txt
MATCH:  1  MISMATCH:  -1  GAPEX:  -1  GAPOP:  -1
1;-1;-1;-1

The local alignment is 54
0 0
TTAATGAAATGATACACGTCTTGCCCCCAAACAGTTGTTTTATTACACGATGTCTGAGACTTGGTGCCTTCACGGGTGCTGATTCCCTAACGTCCCTGTTG-GTGGA-TGG-GATGATCAAGTTCCTAC-G-A-CC-GGGAC-AGCTGG-TC-AGGGGTCC--CCTC--CTTGCTG-G--CCCGGCCGA--ATCCTG-TC-TTCC-CTC-TCGCC-G--AAC--CT-CA--CTTCCTA-A-AAGGAAGTCTGACTCTGTCCTACGCCC-GCGAGGGATC-CGATC-CCTGGTG-T-A-G-GAT----CTGTATCTA---T-CAAG-GT-CT--G-TT-C---CCT--GCCCCCGAC-T-C--CT--CAGGATCAATTTCTTTTGCACTCTTGGAGGCACATCTGCGTATTTC-CCCACACCCTT-CCC-CC-TCT--C---------CCC-TTTTTATATC---C-CCT-TTTT---AT---A---T-C-G------GT-T-TC------GT-
-------------------------------------------------------------------------------------------CAGGCTG--GAGTGGAGTGGCG-TGATCTTG-GCTTACTGCAGCCTTTGCCTCCCCGGCTCGAGCAGTCCTGCCTCAGCCTCCGGAGTAGCTGG--GACCA-CAGGTTCATGCCAC-CATGGCCAGCCAACTTTTGCATGTTTTGTAGAGATGG-GGTCTCACAG

...

Printing contents of following file: BRCAscoreA.txt
#match	mismatch	gap-extend	gap-open
1	-1	-1	-1




Scores contained in: BRCAscoreA.txt
MATCH:  1  MISMATCH:  -1  GAPEX:  -1  GAPOP:  -1
1;-1;-1;-1

The local alignment is 0
0 0
-------TTAA-T-GAAATGATA-C-ACGTCTTGCCCCCAAACAGT-T----GTT-TT-AT-TACAC-GATGTCTG-AGACTTGGTGC-CTTCACGGGTGCTGATTCCCTAACGTCCCTGTTGGTGGATGGGATGATCAAGTTCCTACG-ACCGGGACAGCTGGTC-AGG-GGTCCCCTCCT--TGCTGGCCCGGCCGAATC---CT-G-T--CTT-CCCT-CTCGCCGAACCTCACTTCCTAAAAG-GA-AGTCTGACTCTG-TCCTACGCCCGCGAGGGATCCGA--TCCCTG---GTGTAGG---AT--CTGTATCTAT-CAAGGTCTGTTCCCTGCCCCCGACTCCTCAGGATCAATTTCTTTTG-CA-CTCTTGGAGGCACA-TCTGCGTATTTCCCCACACCCTTCCCCCT--CTCCCCTT-TTTA--T--ATCCCCTTTTTATATCGGTTTCGT
GTACGAGGCAACTGGAAAAAATAGCAATGTATT------AAATAGTGTACGAGTTCTTAATCTCCACAAATTTCCGTTG-C-T-GTGCAATTGAAGGCTG-TGA-AAAC-AA--T--CT-TT-ACAGAT-AG-TGAT-AAG-GACT--GTA-CAGGA-AATTAAACAAGGTGGTGTGAT-ATAAAG-TATCTCGG-AGAA-CAGGTTAGTTAGATTGAGATATTAG--GAAAAGC-CTTTCT-AAAGTGACA-TTTTA-GCTGAT--GA-G---G-G-GGAAAACAATTTAGCTGCCCGTGTTGGTTCA

For penalties, Filename (1) or by hand (2)?  1
Filename:  BRCAscoreA.txt


Printing contents of following file: BRCAscoreA.txt
#match	mismatch	gap-extend	gap-open
1	-1	-1	-1




Scores contained in: BRCAscoreA.txt
MATCH:  1  MISMATCH:  -1  GAPEX:  -1  GAPOP:  -1
1;-1;-1;-1

The global alignment with afine gaps score is -61
0 0
GTACGAGGCAACTGGA-AAAAATAGCAATGTATTAAATAGTGTAC-G-AGTTCTTAATCTCCACAAAT--TTCCGT--TG-CTGTGCAATTGAAG--GCTGTGAAAACAATCTTTACAGATAGTGATAAGGACTGTACAG-GAAATTAAACAAGGTGGTGT-GATATAAAGTATCTC---GGAGAACAGGTTAGTTAGATTGAGATATTAGGAAAAGCCTTTCTA-AAGTGACATTTTAGCTGA-TGAGGGGGAAAACAATT-T-AGCTGCC-CGTGTTGGTTCATCCCTGTACTTGGAAGGCCTAAAGCAGGAAGACTGCCGCAAATCGCAGGCCAGCCCTGGCTACAAAGACTCTGTTTCAAAAAGCCA--AAAAGAT-GA-CTATGA--TT---ATCTAGC-TGGATAGGA-AAGAGCAGCGCAGTGG-C
---C-AGG---CTGGAGTGGAGTGGC---GTGAT-CTTGGCTTACTGCAG-CCTTTGCCTCCCCGGCTCGAGCAGTCCTGCCTCAGCCTCCGGAGTAGCTGGGACCAC-A-GGTT-C----A-TG-CCACCA-TGGCCAGCCAACTTTTGC-ATGTTTTGTAGAGATGGGGTCTCACAGTGTTGCCCAGGCT-GGT--CTCAAACTCCTGGGCTCAGGCGATCCACCTGTCTCAGCCTCCCAGAGTG-CTGGGATTACAATTGTGAGCCACCACGTCCAGCTGGAAGGGTCAACATCTTTTACATTCTGCAA

For penalties, Filename (1) or by hand (2)?  1
Filename:  BRCAscoreA.txt


Printing contents of following file: BRCAscoreA.txt
#match	mismatch	gap-extend	gap-open
1	-1	-1	-1




Scores contained in: BRCAscoreA.txt
MATCH:  1  MISMATCH:  -1  GAPEX:  -1  GAPOP:  -1
1;-1;-1;-1

The local alignment is 13
0 0
GTACGAGGCAACTGGAAAAAATAGCAATGTATTAAATAGTGTACGAGTTCTT--AAT-CTCCA-CAAATTT-CCGTTGCTGTGCAATTGA--AGGCTGTGAAAACAATCTTTACAGA-TAG-T--GA-TA-AGG-AC-TG-TA-CA-GG--A---AA-TTAAACAAGGTGGTGT-GATATAAAGTATCTCGGAGAACAG-GTTAG-TTAGATTGAGATATTAGGAAAAGCCT-TTCTAAAGTGA--CA--T-T-TTAG-CT--GATGAG-G--GGGAAAACAATT-T-AGCTGCC-CGT---G-T---TGGTTCATC-CCTGTACTTGGA-AGGCCTAAAGCAGGAAG-AC-TGCCGCAAATCGCAGGCCAGCCCTGGCTACAAAGACTCTGTTTCAAAAAGCCA--AAAAGAT-GA-CTATGATTATCTAGCTGGATAGGAAA----G-AG-CA-----GCG-C--AGTGG-C
------------------------CAGGCTGGAGTGGAGTG-GCGTGATCTTGGCTTACTGCAGC--CTTTGCC-TCCCCG-GC--TCGAGCAGTC-CTG-CCTC-AGC-CTCCGGAGTAGCTGGGACCACAGGTTCATGCCACCATGGCCAGCCAACTTTTGC-ATGTTTTGTAGAGAT--GGGGTCTC-----ACAGTGTT-GCCCAGGCTG-G-T-CT--CAAACTCCTGGGCTCAGGCGATCCACCTGTCTCAGCCTCCCA-GAGTGCTGGGATTACAATTGTGAGCCACC

In [ ]:
exons = read_single_fasta("BRCA1-exons.txt")
partial = read_single_fasta("BRCA1-partial.txt")

globalalignAF(exons, partial)

In [ ]:
exons = read_single_fasta("BRCA1-exons.txt")
partial = read_single_fasta("BRCA1-partial.txt")

localalign(exons, partial)